In [3]:
import os
from urllib.parse import urljoin
import pystac
from pystac.utils import str_to_datetime
from pystac.extensions.eo import EOExtension
from pystac.extensions.sar import SarExtension
from pystac.extensions.sat import SatExtension
from pystac.collection import Summaries

import requests
from requests.auth import HTTPBasicAuth

from dotenv import load_dotenv
load_dotenv("/home/otto/s1_zarr/.env")

True

In [7]:
import datetime

from pystac import (
    Extent,
    Link,
    Provider,
    ProviderRole,
    RangeSummary,
    SpatialExtent,
    Summaries,
    TemporalExtent,
)
from pystac.extensions.datacube import (
    DatacubeExtension,
    DimensionType,
    HorizontalSpatialDimensionAxis,
)
from pystac.extensions.eo import EOExtension
from pystac.extensions.item_assets import AssetDefinition, ItemAssetsExtension
from pystac.extensions.projection import ProjectionExtension
from pystac.extensions.raster import RasterExtension
from pystac.extensions.sat import SatExtension
from pystac.extensions.scientific import Publication, ScientificExtension
from pystac.extensions.timestamps import TimestampsExtension
from pystac.utils import datetime_to_str, str_to_datetime

In [2]:
username = os.getenv("username_stac_dev")
password = os.getenv("password_stac_dev")

In [ ]:
print(username)
def create_collection():

    """Creation of STAC collection"""
    datacube = {
        "x": {
            "axis": HorizontalSpatialDimensionAxis.X,
            "type": DimensionType.SPATIAL,
            "extent": [20000, 720000],
        },
        "y": {
            "axis": HorizontalSpatialDimensionAxis.Y,
            "type": DimensionType.SPATIAL,
            "extent": [220000, 620000],
        },
        "time": {"type": DimensionType.TEMPORAL, "extent": ["FROM 2011-03-15T00:00:00Z"]},
    }

    parameters = {
        "GL": {
            "description": "global radiation",
            "unit": "W m-2",
        },
        "P0": {
            "description": "mean sea level pressure",
            "unit": "Pa",
        },
        "RH2M": {
            "description": "relative humidity",
            "unit": "percent",
        },
        "RR": {
            "description": "1-hour precipitation sum",
            "unit": "kg m-2",
        },
        "T2M": {
            "description": "air temperature",
            "unit": "degree_Celsius",
        },
        "TD2M": {
            "description": "dew point temperature",
            "unit": "degree_Celsius",
        },
        "UU": {
            "description": "wind speed in eastward direction",
            "unit": "m s-1",
        },
        "VV": {
            "description": "wind speed in northward direction",
            "unit": "m s-1",
        },
    }

    collection = pystac.Collection(
        id="incal-hourly",
        title= "INCA analysis hourly data (1km)",
        description="INCA combines all available datasources - weather stations, remote sensing data, numerical weather forecast models, and a high resolution terrain model - to provide a precise analysis of the condition of the near surface atmosphere.",
        extent=pystac.Extent(
            pystac.SpatialExtent([[7.1, 45.77, 17.74, 49.48]]),
            pystac.TemporalExtent([[str_to_datetime("2011-03-15T00:00:00Z"), None]]),
        ),
        keywords=["GeoSphere", "INCA", "radiation", "pressure", "humiditiy", "precipitation", "temperature", "wind speed"],
        license="CC-BY-4.0",
        extra_fields={"cube:dimensions": datacube,
                      "cube:variables": parameters}, 
    )

    collection.providers = [
    pystac.Provider(
        name="EODC",
        roles=[
            pystac.ProviderRole.HOST,
        ],
        url="https://eodc.eu/",
        ),
    pystac.Provider(
        name="GeoSphere Austria",
        roles=[
            pystac.ProviderRole.PRODUCER,
            pystac.ProviderRole.LICENSOR,
            pystac.ProviderRole.PROCESSOR,
        ],
        url="https://data.hub.geosphere.at/",
        ),        
    ]

    collection.stac_extensions = [
        SarExtension.get_schema_uri(),
        SatExtension.get_schema_uri(),
        EOExtension.get_schema_uri(),
        DatacubeExtension.get_schema_uri(),
    ]

    summaries_dict = {}
    summaries_dict["grid"] = ["1x1km"]
    summaries_dict["temporal resolution"] = ["hourly"]
    summaries_dict["epsg"] = ["31287"]
    summaries_dict["projection"] = ["MGI / Austria Lambert (EPSG: 31287)"]
    summaries_dict["doi"] = ["https://doi.org/10.60669/6akt-5p05"]
    summaries_dict["timezone"] = ["UTC"]
    collection.summaries = Summaries(summaries_dict)





    destination = "simon_demo/staccoll"
    collection_path = os.path.join(destination, "{}.json".format(collection.id))
    collection.set_self_href(collection_path)
    collection.save_object()

    collection.add_asset(
    key="thumbnail",
    asset=pystac.Asset(
        href="https://titiler.services.eodc.eu/cog/preview?url=https%3A%2F%2Fobjectstore.eodc.eu%3A2222%2F88346baf22914e828ad2c1763e5e01ff%3Acoverage-maps%2FSENTINEL1_GRD_COVERAGE_2016.tif&nodata=0&rescale=0%2C251&colormap_name=matter",  # URL or path to your thumbnail
        media_type=pystac.MediaType.PNG,
        roles=["thumbnail"],
        title="Thumbnail"
    )
)
    return collection

## Create Collection
create_collection()
k = create_collection()

k.to_dict()


app_host = "https://dev.stac.eodc.eu/ingestion/v1"
url = urljoin(app_host, "/collections")



r = requests.post(app_host + "/collections", json=k.to_dict(), auth=HTTPBasicAuth(username, password), timeout = 30)
#r = requests.delete("https://dev.stac.eodc.eu/ingestion/v1/collections/incal-hourly", auth=HTTPBasicAuth(username, password), timeout = 30)

print(r.text)


ingest1
{"id":"incal-hourly","description":"INCA combines all available datasources - weather stations, remote sensing data, numerical weather forecast models, and a high resolution terrain model - to provide a precise analysis of the condition of the near surface atmosphere.","stac_version":"1.1.0","links":[{"rel":"items","type":"application/geo+json","href":"https://dev.stac.eodc.eu/ingestion/v1/collections/incal-hourly/items"},{"rel":"parent","type":"application/json","href":"https://dev.stac.eodc.eu/ingestion/v1/"},{"rel":"root","type":"application/json","href":"https://dev.stac.eodc.eu/ingestion/v1/"},{"rel":"self","type":"application/json","href":"https://dev.stac.eodc.eu/ingestion/v1/collections/incal-hourly"}],"stac_extensions":["https://stac-extensions.github.io/sar/v1.0.0/schema.json","https://stac-extensions.github.io/sat/v1.0.0/schema.json","https://stac-extensions.github.io/eo/v1.1.0/schema.json","https://stac-extensions.github.io/datacube/v2.2.0/schema.json"],"title":"INC